In [1]:
!pip install git+https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git    

  Cloning https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git to /tmp/pip-req-build-lzowx0sd
  Created wheel for auspixdggs: filename=auspixdggs-1.0.0-py3-none-any.whl size=69769 sha256=0c10ed565841c35c24232de08a2b75bf138c8dd228e3cc5b09a2efa4f87dfa6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-wb2w2650/wheels/06/73/68/673cc96e3ec5fd40aefc75b28dd3f8daf72643236485ac32b9
Successfully built auspixdggs


In [2]:
!pip install geojson shapely pygeoj matplotlib

In [3]:
import geojson
fname = "SA_RoadsExample_lines.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

In [4]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -34.74
y_coord = 138.78

map1 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map1.add_layer(geo_json1)
map1

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [16]:
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from geojson.utils import coords
from geojson import Feature, FeatureCollection
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     return ret
    
def geojson_to_shape(g):
    return shape(g)
    
def get_cells_in_geojson(geojson, resolution):
    list_cells = []
    for fea in geojson['features']:  # for feature in attribute table
        res_cells = get_cells_in_feature(fea, resolution)
        list_cells = list(list_cells + res_cells)
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = res_cells
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]

    return cells


DGGS_RESOLUTION=8
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION)  
list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

#uncomment if you want to get a file output
#geojson_dump = geojson.dumps(feature_collection, indent=4, sort_keys=True)
#with open('poly.geojson', 'w') as json_file:
#    data = geojson.dump(feature_collection, json_file, indent=4, sort_keys=True)        

In [17]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'red', 
                          'fillOpacity': 0.3})
map2.add_layer(geo_json2)
map2.add_layer(geo_json1)
map2

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [18]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [33]:
target_feature_type = 'asgs16_sa2'

set_of_sa2_features = set([])
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id'] + "00"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_sa2_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


Processing R7751215200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751218400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751218500
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751223800
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224600
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751226200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751226500
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751218400
Adding http://linked.data.gov.au

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751227400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751227400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751227400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751224300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751223100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227300
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224600
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223500
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227300
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223500
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751226100
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751218200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751218200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751218500
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751224700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751227000
Adding http://linked.data.gov.au/dataset/asgs2016/stati

...

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751226300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751226400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223600
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751215800
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224600
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223800
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751227000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223800
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751223600
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751226000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751226100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751215200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751215800
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224100
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751218400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751215700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751215700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223000
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751223800
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751226200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751226200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751218500
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049
Processing R7751226200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751226100
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751215200
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227000
Adding http://linked.data.gov.au/dataset/asgs2016/stati

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751224700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751224400
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751227300
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003
Processing R7751215700
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751223800
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033
Processing R7751224600
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033


In [34]:
print(json.dumps(list(set_of_sa2_features), indent=3))

[
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402021033",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/401021003",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel2/402051049"
]


In [35]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data




In [36]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [37]:
for feature_uri in list(set_of_sa2_features):
    curr_geojson_layer = GeoJSON(data=get_geom(feature_uri), 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)


In [38]:
map3.add_layer(geo_json1)



In [39]:
map3

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…